# Bokeh Demos

*The "User Guide" linked on this page is a very good resource :* https://bokeh.pydata.org/en/latest/

## Part 1

*An example with a scatter plot and connected table plus a callback to allow access to the selected values.*


## 1.1 Import the necessary libraries



In [ ]:
# Import needed libraries.
import pandas as pd
from bokeh.plotting import *
from bokeh.layouts import row, column
from bokeh.models import ColumnDataSource, Scatter, Select, CustomJS
from bokeh.models.widgets import DataTable, TableColumn

output_notebook()
# if you uncomment the line below, the plot will be exported to an html file
#output_file("scatterSelect.html", title='scatter')

## 1.2 Read in the data

*I am using exoplanet data from the [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/). A description of each column is provided at the top of the file.*

In [ ]:
# Read in (or create) data.
df = pd.read_csv('PS_2021.10.05_11.19.37.csv', comment='#')

# for this example, I will only keep rows that have values for mass and radius
usedf = df.loc[ (pd.notnull(df['pl_bmasse'])) & (pd.notnull(df['pl_rade']))].reset_index()
usedf

## 1.3 Define the "ColumnDataSource" and the plots.

*A ColumnDataSource will hold a python dictionary (or a panda dataframe) containing your data and can be accessed by Bokeh.*

*I'm going to wrap these commands in a function so that I can access them later (needed while creating the callback below).*

In [ ]:
# create a column data source containing the mass and radius
source = ColumnDataSource(data=dict(x=usedf['pl_bmasse'], y=usedf['pl_rade']))
    
def createPlot(source):

    # define the tools you want to use
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"

    # create a new plot and renderer
    f = figure(tools=TOOLS, width=350, height=350, title=None, x_axis_type='log', y_axis_type='log', 
               y_range=(0.3, 50), x_range=(0.03, 1e5))
    renderer = f.scatter('x', 'y', source=source, color='black', alpha=0.5, size=5, marker='circle')
    f.xaxis.axis_label = 'mass [Earth masses]'
    f.yaxis.axis_label = 'radius [Earth radii]'

    # (optional) define different colors for highlighted and non-highlighted markers
    renderer.selection_glyph = Scatter(fill_alpha=1, fill_color="firebrick", line_color=None)
    renderer.nonselection_glyph = Scatter(fill_alpha=0.2, fill_color="gray", line_color=None)

    return f

f = createPlot(source)

# show the plot
show(f)

## 1.4 Add a table 

*Selecting data on the Bokeh DataTable will highlith it on the scatter plot and vice versa.*

*Again, I am going to wrap this in a function so that I can use it later.*

In [ ]:
# I want to send the labels to this function also so that it can be more versatile
# I will expect the labes to be a dict with a key for each columns that I want to include and value for the label
def createTable(source, labels, w=350, h=300):
    # create a table to hold the selections
    columns = []
    for field in labels:
        columns.append(TableColumn(field=field, title=labels[field]))

    t = DataTable(source=source, columns=columns, width=w, height=h)

    return t

t = createTable(source, dict(x="mass [Earth masses]", y="radius [Earth radii]"))

# create a griplot layout and show the plot and table
layout = gridplot([[f, t]])

show(layout)

## 1.5 Add a "callback" to get the selected indices for later use in the notebook

*A callback is a generic term for a function that is called after some event happens.  Here the function will be called after a slider's value is changed.*

*Bokeh can work with callbacks in either Python of Javascript.  When working fully within a Jupyter notebook, it probably makes most sense to write callbacks in Python.  However, if you want to export your interactive plot to a website, you would need to write a Javascript callback.  I will show an example of the Javascript version below for completeness.*

## 1.5a The Python approach:

*In this example below, I am writing the callback function in Python.  This approach has the benefit of already being in a language you know (Python), but you cannot use Python callbacks to create a .html file.  Only Javascript callbacks can be used to create an interactive plot for your website.  (Also Python callbacks will not work in colab.)*  

In [ ]:
# It appears that in order for the Python callback to work, I need to redefine the plot and table within this cell
source = ColumnDataSource(data=dict(x=usedf['pl_bmasse'], y=usedf['pl_rade']))
f = createPlot(source)
t = createTable(source, dict(x="mass [Earth masses]", y="radius [Earth radii]"))

# define a global variable the will be modified within the callback
indices = []
def selectionHandler(attr,old,new):
    global indices
    indices = source.selected.indices

# attach the callback to the data source to be run when the selection indices change
source.selected.on_change("indices", selectionHandler)

# create a griplot layout and show the plot and table
layout = gridplot([[f, t]])

# in order to run a Python callback in a Jupyter notbook, you need to include the following
def bkapp(doc):
    doc.add_root(layout)

show(bkapp)

In [ ]:
# test that we have access to the selected points
usedf.iloc[list(indices)]#['pl_bmasse']

## 1.5b The Javascript approach:

*In this example below, I am writing the callback function in Javascript.  This will allow us to show the result within a colab notebook and also to save the resulting plot as a standalone .html file that could be used on your website.  (But, you need to learn a little Javascript.)*

In [ ]:
# if you ran the python version above, you will need to recreate the plot
source = ColumnDataSource(data=dict(x=usedf['pl_bmasse'], y=usedf['pl_rade']))
f = createPlot(source)
t = createTable(source, dict(x="mass [Earth masses]", y="radius [Earth radii]"))

# define the javascript callback using Bokeh's CustomJS
#I will wrap this in a function so that I can use it later
def attachSelectionHandlerJS(source):
    selectionHandlerJS = CustomJS(args=dict(s1=source), code="""
        //get the indices
        const indices = cb_obj.indices;

        //execute a command in this notebook to set the indices
        IPython.notebook.kernel.execute("indices = " + indices);
    """)

    # attach the callback to the data source to be run when the selection indices change
    source.selected.js_on_change("indices", selectionHandlerJS)

attachSelectionHandlerJS(source)

# create a griplot layout and show the plot and table
layout = gridplot([[f, t]])
    
show(layout)


In [ ]:
# test that we have access to the selected points
usedf.iloc[list(indices)]#['pl_bmasse']

## Part 2

*A scatter plot with dropdown widgets to change the data in the plot.*

## 2.1 Adding a dropdown widget

*I will work with the same Pandas DataFrame, but now I want to allow the user to be able to interactively select the data to plot on each axis from a few different columns.  First I will need to create a new ColumnDataSource.  Then I can use the function from above the create the plot.  Finally I will create the dropdowns and add them to the plot.* 

In [ ]:
# I will follow a very similar method as before but I will provide more columns to the ColumnDataSource
usedf = df.loc[ (pd.notnull(df['pl_bmasse'])) & (pd.notnull(df['pl_rade'])) & 
                (pd.notnull(df['pl_orbeccen'])) & (pd.notnull(df['pl_orbper'])) &
                (pd.notnull(df['st_teff'])) & (pd.notnull(df['sy_vmag']))].reset_index()
source = ColumnDataSource(data=dict(x=usedf['pl_bmasse'], y=usedf['pl_rade'], 
                                    mass=usedf['pl_bmasse'],
                                    rad=usedf['pl_rade'],
                                    ecc=usedf['pl_orbeccen'],
                                    per=usedf['pl_orbper'],
                                    teff=usedf['st_teff'],
                                    vmag=usedf['sy_vmag']))

# use the function from above the create the plot
f = createPlot(source)

#create a dict to hold all the keys and labels that I will want to use
labels = dict(mass="mass [Earth masses]",
              rad="radius [Earth radii]",
              ecc="eccentricity",
              per="orbital period [days]",
              teff="star Teff [K]",
              vmag="star V [mag]")
    
# Again, I will wrap this in a function
def createDropdowns(source, f, labels):
    # define widgets to change the x and y values to plot
    
    # I will create a few arrays here
    # "options" will be drawn from labels and will contain the text that I want to show up in the dropdowns
    # "keys" will be drawn from labels and will contain the actual key values that I defined in the source dict
    # "bounds" will contain axis limits for each key, note that since I'm using log scaling, I need to make these >0
    # (in principle this could be done as a single dict, but having the lists makes the javascript side easier)
    options = list(labels.values())
    keys = list(labels.keys())
    bounds = [];
    for k in labels:
        bounds.append([max(0.5*min(source.data[k]), 0.0001), max(2*max(source.data[k]), 0.0001)])
    xSelect = Select(title="x axis", value=options[0], options=options)
    ySelect = Select(title="y axis", value=options[1], options=options)

    # Javascript callback
    # I'm going to create a single callback to handle both axes
    callback = CustomJS(args=dict(source=source, keys=keys, options=options, bounds = bounds,
                                 axes={"x":f.xaxis[0], "y":f.yaxis[0]}, 
                                 ranges={"x":f.x_range, "y":f.y_range} ), 
                        code="""
        //get the value from the dropdown 
        //Note: "this" is like Python's "self"; here it will containt the select element.
        var val = this.value;

        //now find the index within the options array so that I can find the correct key to use
        var index = options.indexOf(val);
        var key = keys[index];

        //check with axis this is
        var ax = "x";
        if (this.title == "y axis") ax = "y";
        console.log(this.title, ax)

        //change the data is being plotted
        source.data[ax] = source.data[key];
        source.change.emit();

        //change the axis label
        axes[ax].axis_label = val;

        //change the bounds
        ranges[ax].start = bounds[index][0];
        ranges[ax].end = bounds[index][1];

    """)
    xSelect.js_on_change("value", callback)
    ySelect.js_on_change("value", callback)
    
    return xSelect, ySelect
   
xSelect, ySelect = createDropdowns(source, f, labels)

layout = row(
    f,
    column(xSelect,ySelect)
)

# show the plot
show(layout)


## 2.2 Add the table back in

*I can use most of the code from above, but I will increase the columns in the table.*

In [ ]:
# use the function from above the create the plot
source = ColumnDataSource(data=dict(x=usedf['pl_bmasse'], y=usedf['pl_rade'], 
                                    mass=usedf['pl_bmasse'],
                                    rad=usedf['pl_rade'],
                                    ecc=usedf['pl_orbeccen'],
                                    per=usedf['pl_orbper'],
                                    teff=usedf['st_teff'],
                                    vmag=usedf['sy_vmag']))

f = createPlot(source)

t = createTable(source, labels, w=800)

xSelect, ySelect = createDropdowns(source, f, labels)

attachSelectionHandlerJS(source)

layout = column(
    row(column(xSelect,ySelect), f,),
    row(t)
)

# show the plot
show(layout)

In [ ]:
# test that we have access to the selected points
usedf.iloc[list(indices)]#['pl_orbeccen']